Запрос к 2гис api и маршрутизация

Задача: потыкать 2gis api маршрутизации и понять, возможно ли получать несколько маршрутов между 2-мя точками

In [45]:
import pandas as pd
import geopandas as gpd

In [3]:
url = 'https://routing.api.2gis.com/routing/7.0.0/global?key=ddd69dc6-ad5d-4412-ba72-fcc1e8ce921d'

In [105]:
import requests
import json

API_URL = "http://routing.api.2gis.com/routing/7.0.0/global"
API_KEY = "34fc9584-ade7-4bdc-999f-f882a498cc0a"

headers = {
    "Content-Type": "application/json"
}

body = {
  "filters": [],
  "locale": "ru",
  "points": [
    {
      "lon": 37.582147949213654,
      "lat": 55.71696885746129,
      "type": "walking"
    },
    {
      "lon": 37.60585633975072,
      "lat": 55.714261701646154,
      "type": "walking"
    }
  ],
  "transport": "walking",
  "options": [
    "pedestrian_instructions"
  ],
  "route_mode":"fastest",
  "alternative": 2
}
params = {
    "key": API_KEY
}

response = requests.post(API_URL, params=params, headers=headers, json=body)

if response.status_code == 200:
    result = response.json()
    print(json.dumps(result, indent=2, ensure_ascii=False))
else:
    print(f"Ошибка {response.status_code}: {response.text}")


{
  "message": null,
  "query": {
    "alternative": 2,
    "filters": [],
    "locale": "ru",
    "options": [
      "pedestrian_instructions"
    ],
    "points": [
      {
        "lat": 55.71696885746129,
        "lon": 37.58214794921366,
        "type": "walking"
      },
      {
        "lat": 55.71426170164615,
        "lon": 37.60585633975072,
        "type": "walking"
      }
    ],
    "route_mode": "fastest",
    "transport": "walking"
  },
  "result": [
    {
      "algorithm": "кратчайший",
      "begin_pedestrian_path": {
        "geometry": {
          "selection": "LINESTRING(37.582147 55.716968, 37.581213 55.717427)"
        }
      },
      "end_pedestrian_path": {
        "geometry": {
          "selection": "LINESTRING(37.605883 55.714261, 37.605856 55.714261)"
        }
      },
      "id": "7898480947999084752",
      "maneuvers": [
        {
          "comment": "start",
          "icon": "start",
          "id": "10039539491242512218",
          "outcoming_path"

In [106]:
# посмотрим что из себя представляют данные
df = pd.json_normalize(response.json()['result'])
df


,algorithm,id,maneuvers,reliability,route_id,total_distance,total_duration,type,ui_total_duration,waypoints,begin_pedestrian_path.geometry.selection,end_pedestrian_path.geometry.selection,ui_total_distance.unit,ui_total_distance.value
0,кратчайший,7898480947999084752,"[{'comment': 'start', 'icon': 'start', 'id': '...",1.0,moscow-pd-back.m1/pedestrianrouting/1759618684...,3119,2079,pedestrianrouting,34 мин,"[{'original_point': {'lat': 55.71742749506007,...","LINESTRING(37.582147 55.716968, 37.581213 55.7...","LINESTRING(37.605883 55.714261, 37.605856 55.7...",км,3.1
1,по основным улицам,400443975899736026,"[{'comment': 'start', 'icon': 'start', 'id': '...",1.0,moscow-pd-back.m1/pedestrianrouting/1759618684...,3520,2346,pedestrianrouting,39 мин,"[{'original_point': {'lat': 55.71742749506007,...","LINESTRING(37.582147 55.716968, 37.581213 55.7...","LINESTRING(37.605883 55.714261, 37.605856 55.7...",км,3.5


In [101]:
# преблема в отдельных кусочках
df.iloc[0]['maneuvers']

[{'comment': 'start',
  'icon': 'start',
  'id': '5549043055644061043',
  'outcoming_path': {'distance': 22,
   'duration': 15,
   'geometry': [{'length': 22,
     'selection': 'LINESTRING(37.182166 55.716939, 37.181826 55.716864)',
     'style': 'normal',
     'zlevel': 'zlevel-normal'}],
   'names': []},
  'outcoming_path_comment': '20 м прямо',
  'type': 'pedestrian_begin'},
 {'attribute': 'empty',
  'comment': 'Поворот налево',
  'icon': 'crossroad_left',
  'id': '7944772796427719863',
  'outcoming_path': {'distance': 16,
   'duration': 10,
   'geometry': [{'length': 16,
     'selection': 'LINESTRING(37.181826 55.716864, 37.181889 55.716773, 37.181896 55.716720)',
     'style': 'normal',
     'zlevel': 'zlevel-normal'}],
   'names': []},
  'outcoming_path_comment': '20 м прямо',
  'turn_angle': -78,
  'turn_direction': 'left',
  'type': 'pedestrian_crossroad'},
 {'attribute': 'empty',
  'comment': 'Поворот налево',
  'icon': 'crossroad_left',
  'id': '11909329134213535440',
  'outc

In [102]:
list_lines = []
index = 0
for i in df['maneuvers']:
    total_length_i = 0
    list_i = []
    for j in i:
        if 'outcoming_path' not in j:
            # print(j)
            continue
        try:
            kgkgn = j['outcoming_path']['geometry']
            for q in kgkgn:
                # для проверки сравним с данными из атрибутов маршрутов
                total_length_i += q['length']
                list_i.append(q['selection'])
        except KeyError:
            print('Ошибка')
    list_lines.append(list_i)
    print(f'Длина при суммировании: {total_length_i}')
    print(f"Длина исходная: {df.iloc[index]['total_distance']}")
    index += 1



Длина при суммировании: 33158
Длина исходная: 32915
Длина при суммировании: 35594
Длина исходная: 35386


In [103]:
from shapely import wkt
from shapely.geometry import MultiLineString

# формируем конечные мультилинии
all_multilines = []
for line in list_lines:
    line_geoms = [wkt.loads(l) for l in line]
    multi_line = MultiLineString(line_geoms)
    all_multilines.append(multi_line.wkt)
    
all_multilines

['MULTILINESTRING ((37.182166 55.716939, 37.181826 55.716864), (37.181826 55.716864, 37.181889 55.716773, 37.181896 55.71672), (37.181896 55.71672, 37.183 55.716771, 37.183745 55.716805), (37.183745 55.716805, 37.183412 55.717585, 37.183258 55.717865, 37.183031 55.718192, 37.182868 55.71842, 37.182735 55.718585, 37.182717 55.718607, 37.182347 55.719065, 37.182266 55.719164, 37.182083 55.71939, 37.181873 55.719617, 37.181551 55.719927, 37.180996 55.720463, 37.180512 55.720928, 37.180335 55.721099), (37.180335 55.721099, 37.180445 55.721135, 37.179904 55.721657, 37.179811 55.721763, 37.179746 55.721836, 37.179644 55.721972, 37.179519 55.722186, 37.179461 55.722285, 37.179357 55.722463, 37.17868 55.723626), (37.17868 55.723626, 37.1789 55.723725, 37.184728 55.724121, 37.186946 55.72386, 37.187014 55.724045), (37.187014 55.724045, 37.187064 55.724183, 37.187794 55.724097, 37.189707 55.723965, 37.189836 55.72398, 37.190036 55.724164, 37.190287 55.724077), (37.190287 55.724077, 37.190495 55.

In [104]:
# посмотрим на линии в вебе
gdf = gpd.GeoDataFrame(data=df, geometry=gpd.GeoSeries.from_wkt(all_multilines), crs=4326)
gdf.explore()

In [96]:
# Проверим длину, переведя в UTM зону
gdf.to_crs('32637').geometry.length

0    33303.499749
1    35714.711624
dtype: float64